In [1]:
import numpy as np
from TidalPy.RadialSolver import build_rs_input_homogeneous_layers, build_rs_input_from_data
from TidalPy.rheology import Maxwell, Elastic

## `build_rs_input_homogeneous_layers` Performance

**v0.6.0:**

slices_per_layer = 5; perform_checks=False; num_layers:
- 1:   32.2us
- 5:   33.4us
- 10:  35.8us
- 15:  37.7us
- 20:  39.5us
- 50:  52us
- 100: 74us

slices_per_layer = 5; perform_checks=True; num_layers:
- 1:   32.5us
- 5:   34.3us
- 10:  35.7us
- 15:  38.7us
- 20:  40.6us
- 50:  54us
- 100: 75.3us

slices_per_layer = 50; perform_checks=True; num_layers:
- 1:   34.9us
- 5:   47.8us
- 10:  65.4us
- 15:  82.0us
- 20:  95.0us
- 50:  188.0us
- 100: 339.0us

**v0.6.1:**
slices_per_layer = 5; perform_checks=True; num_layers:
- 1:   9.09 μs
- 5:   10.6 μs
- 10:  12.3 μs
- 15:  14.2 μs
- 20:  16 μs
- 50:  26.5 μs
- 100: 43.5 μs

slices_per_layer = 50; perform_checks=True; num_layers:
- 1:   11.5 μs
- 5:   22.4 μs
- 10:  36 μs
- 15:  49.2 μs
- 20:  62.6 μs
- 50:  144 μs
- 100: 287 μs

In [3]:
slices_per_layer = 50
perform_checks = True

for num_layers in (1, 5, 10, 15, 20, 50, 100):
    planet_radius = 1.0e6
    forcing_frequency = 1.0e-4
    density_tuple = tuple([3000.0 for i in range(num_layers)])
    static_bulk_modulus_tuple = tuple([1.0e10 for i in range(num_layers)])
    static_shear_modulus_tuple = tuple([1.0e10 for i in range(num_layers)])
    bulk_viscosity_tuple = tuple([1.0e30 for i in range(num_layers)])
    shear_viscosity_tuple = tuple([1.0e30 for i in range(num_layers)])
    layer_type_tuple = tuple(['solid' for i in range(num_layers)])
    layer_is_static_tuple = tuple([False for i in range(num_layers)])
    layer_is_incompressible_tuple = tuple([False for i in range(num_layers)])
    shear_rheology_model_tuple = tuple([Maxwell() for i in range(num_layers)])
    bulk_rheology_model_tuple = tuple([Elastic() for i in range(num_layers)])
    thickness_fraction_tuple = tuple([1.0 / num_layers for i in range(num_layers)])

    inputs = (
        planet_radius,
        forcing_frequency,
        density_tuple,
        static_bulk_modulus_tuple,
        static_shear_modulus_tuple,
        bulk_viscosity_tuple,
        shear_viscosity_tuple,
        layer_type_tuple,
        layer_is_static_tuple,
        layer_is_incompressible_tuple,
        shear_rheology_model_tuple,
        bulk_rheology_model_tuple
    )

    # Make sure it works once
    print(f"Working on #Layers = {num_layers}.")
    output = build_rs_input_homogeneous_layers(*inputs, thickness_fraction_tuple=thickness_fraction_tuple, slice_per_layer=slices_per_layer, perform_checks=perform_checks)

    %timeit build_rs_input_homogeneous_layers(*inputs, thickness_fraction_tuple=thickness_fraction_tuple, slice_per_layer=slices_per_layer, perform_checks=perform_checks)

Working on #Layers = 1.
11.5 μs ± 78.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Working on #Layers = 5.
22.4 μs ± 192 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 10.
36 μs ± 360 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 15.
49.2 μs ± 235 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 20.
62.6 μs ± 333 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 50.
144 μs ± 608 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 100.
287 μs ± 13.1 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## `build_rs_input_from_data` Performance
**v0.6.0:**
slices_per_layer = 5; perform_checks=True; num_layers:
- 1:   145us
- 5:   586us
- 10:  1.15ms
- 15:  1.7ms
- 20:  2.29ms
- 50:  5.89ms
- 100: 12ms

slices_per_layer = 50; perform_checks=True; num_layers:
- 1:   153 μs
- 5:   658 μs
- 10:  1.28 ms
- 15:  1.92 ms
- 20:  2.51 ms
- 50:  6.58 ms
- 100: 13.8 ms

**v0.6.1:**
slices_per_layer = 5; perform_checks=True; num_layers:
- 1:   13.4 μs 
- 5:   16.3 μs
- 10:  19.9 μs
- 15:  24.6 μs
- 20:  27.7 μs
- 50:  48.7 μs
- 100: 83.6 μs

slices_per_layer = 50; perform_checks=True; num_layers:
- 1:   18.6 μs
- 5:   42.2 μs
- 10:  71.5 μs
- 15:  101 μs
- 20:  129 μs
- 50:  302 μs
- 100: 595 μs

In [4]:
slices_per_layer = 5
perform_checks = True
warnings = False

for num_layers in (1, 5, 10, 15, 20, 50, 100):
    planet_radius = 1.0e6
    forcing_frequency = 1.0e-4
    layer_thickness = planet_radius / num_layers
    dr = layer_thickness / slices_per_layer

    # The dr addition below ensures there is a small error in the radius array that the helper has to fix
    radius_array = np.concatenate([np.linspace(layer_thickness * i + 0.1 * dr, layer_thickness * (i + 1), slices_per_layer) for i in range(num_layers)])
    density_array = np.linspace(10_000.0, 4_000.0, radius_array.size)
    static_bulk_modulus_array = np.linspace(1000.0e9, 250.0e9, radius_array.size)
    static_shear_modulus_array = np.linspace(250.0e9, 50.0e9, radius_array.size)
    bulk_viscosity_array = np.logspace(30, 26, radius_array.size)
    shear_viscosity_array = np.logspace(30, 18, radius_array.size)
    
    layer_upper_radius_tuple = tuple([(i+1) * layer_thickness for i in range(num_layers)])
    layer_type_tuple = tuple(['solid' for i in range(num_layers)])
    layer_is_static_tuple = tuple([False for i in range(num_layers)])
    layer_is_incompressible_tuple = tuple([False for i in range(num_layers)])
    shear_rheology_model_tuple = tuple([Maxwell() for i in range(num_layers)])
    bulk_rheology_model_tuple = tuple([Elastic() for i in range(num_layers)])
    thickness_fraction_tuple = tuple([1.0 / num_layers for i in range(num_layers)])

    inputs = (
        forcing_frequency,
        radius_array,
        density_array,
        static_bulk_modulus_array,
        static_shear_modulus_array,
        bulk_viscosity_array,
        shear_viscosity_array,
        layer_upper_radius_tuple,
        layer_type_tuple,
        layer_is_static_tuple,
        layer_is_incompressible_tuple,
        shear_rheology_model_tuple,
        bulk_rheology_model_tuple
    )

    # Make sure it works once
    print(f"Working on #Layers = {num_layers}.")
    output = build_rs_input_from_data(*inputs, perform_checks=perform_checks, warnings=warnings)

    %timeit build_rs_input_from_data(*inputs, perform_checks=perform_checks, warnings=warnings)

Working on #Layers = 1.
13.4 μs ± 126 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Working on #Layers = 5.
16.3 μs ± 64.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Working on #Layers = 10.
19.9 μs ± 169 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
Working on #Layers = 15.
24.6 μs ± 436 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 20.
27.7 μs ± 1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 50.
48.7 μs ± 334 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
Working on #Layers = 100.
83.6 μs ± 514 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
